# Herbert - Polish RoBERT'a model 


## Prepare the data.

* Wikipedia, Link: 
* Oscar
* Books, Polish "Wolne lektury"



Usefull resources


* https://github.com/pytorch/fairseq/blob/master/examples/roberta/README.pretraining.md
* https://github.com/musixmatchresearch/umberto/issues/2


Get raw text from different sources and concat in one big data file. 

Usefull shell commands:


Move files to another directory,  where isbn's are in the file list (wolne lektury non polish)

```
cat wolne_lektury_non_polish_isbn.txt | xargs -I{} sh -c "mv *'{}'* ./non_polish/;" 
```


Cat all text files and instert new line between each text

```sh
find *content.txt | xargs -I{} sh -c "cat '{}'; echo ''" > corpus_[type]_[date].txt

```

Take 11768022 first lines form splited wikipedia file
```
head -11768022 corpus_wiki_2020-02-13.txt > corpus_wiki_2020-02-13_sample.txt
```


In [1]:
import csv
import sys
import datetime as dt
import os
from pathlib import Path
import re
import tqdm as tq
from tqdm.notebook import tqdm
import mmap

print("import")

import


In [2]:
# utils functions


def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

Read wikipedia csv file and save in txt.

Run only once, just for generating file!!!

In [4]:

wikipedia_path='./data/wikipedia_2020-02-01.csv'
output_path='./data/corpus_wikipedia_2020-02-01.txt'

csv.field_size_limit(sys.maxsize)

t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(wikipedia_path) as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for article in csv_reader:
            text = article['CleanText']

            output_file.write(text)
            # put new line of the end of the article
            output_file.write('\n')   
t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')   

Read book dataset and normalize line splitiing. The textfile has '\n' in middle of the sentence. It is not necessary if you have proper file.

Input: concatenated book textfile
Output: file with removed new lines in the middle of the sentence. 

Run once!!!


In [3]:


input_path='./data/corpus_books_2020_02_24.txt'

p = Path(input_path)
print(p.with_suffix(''))
output_path = f"{p.with_suffix('')}_fix.txt"

print(output_path)

# reg = re.compile('(?<!\.)(\n)(?=[a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ])',re.MULTILINE )
# reg = re.compile('(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )
#reg = re.compile('(?<!\.\n)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )


# remove line breaks in the middle of the sentence
reg = re.compile('(?<=[A-Za-ząćęłńóśźż,—-])(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )
#replace it by space
rep_lines=' '

# replace many dots in lines with one dot and line break
# https://regex101.com/r/qCTEPu/1
reg_dots = re.compile(r"^[\.-]([\.\s]*\n)",re.MULTILINE)
rep_dots='\n'

# https://pymotw.com/3/mmap/#regular-expressions

line_buff = 10007# 10007 # this is prime number, next is  10009  10037 
N = 0 

def save_buffer2file(output_file, text):

    # proces and write
    replace_text = reg.sub(rep_lines, text)
    #clean lines with only one character (dots, etc)
    replace_text = reg_dots.sub(rep_dots, replace_text)

    output_file.write(replace_text)



t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(input_path) as f:

        text=''
        for line in tqdm(f,total=get_num_lines(input_path)):
            # get block of file (line_buff) lines and replace 
            if N<line_buff:
                # glue lines
                text+=line
                N+=1
            else:
                save_buffer2file(output_file, text)
                
                text=''
                N=0
                
        # for the rest of the file proces and write
        if N>0:
            save_buffer2file(output_file, text)
            

t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')  

data/corpus_books_2020_02_24
data/corpus_books_2020_02_24_fix.txt



Done. Takes=0:07:03.430978


## Create vocabulary


### Prepare data for vocab

Create files for training vocabulary. One sentence per line, use polish sentence tokenizer. 
Sentence piece model is capable of handling around 12_000_000 sentences, so more sentences is not necessary.





In [3]:
import nltk

#nltk.download()
#nltk.download('punkt')

extra_abbreviations = ['ps',  'inc', 'Corp', 'Ltd', 'Co', 'pkt', 'Dz.Ap', 'Jr', 'jr', 'sp', 'Sp', 'poj',  'pseud', 'krypt', 'sygn', 'Dz.U', 'ws', 'itd', 'np', 'sanskryt', 'nr', 'gł', 'Takht', 'tzw', 't.zw', 'ewan', 'tyt', 'oryg', 't.j', 'vs', 'l.mn', 'l.poj' ]

position_abbrev = ['Ks', 'Abp', 'abp','bp','dr', 'kard', 'mgr', 'prof', 'zwycz', 'hab', 'arch', 'arch.kraj', 'B.Sc', 'Ph.D', 'lek', 'med', 'n.med', 'bł', 'św', 'hr', 'dziek' ]

roman_abbrev= [] #['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XII','XIV','XV','XVI', 'XVII', 'XVIII','XIX', 'XX', 'XXI' ]

quantity_abbrev = [ 'mln', 'obr./min','km/godz', 'godz', 'egz', 'ha', 'j.m', 'cal', 'obj', 'alk', 'wag' ] # not added: tys.

actions_abbrev = ['tłum','tlum','zob','wym', 'pot', 'ww', 'ogł', 'wyd', 'min', 'm.i', 'm.in', 'in', 'im','muz','tj', 'dot', 'wsp', 'właść', 'właśc', 'przedr', 'czyt', 'proj', 'dosł', 'hist', 'daw', 'zwł', 'zaw' ]

place_abbrev = ['Śl', 'płd', 'geogr']

lang_abbrev = ['jęz','fr','franc', 'ukr', 'ang', 'gr', 'hebr', 'czes', 'pol', 'niem', 'arab', 'egip', 'hiszp', 'jap', 'chin', 'kor', 'tyb', 'wiet', 'sum']

military_abbrev = ['kpt', 'kpr', 'obs', 'pil', 'mjr','płk', 'dypl', 'pp', 'gw', 'dyw', 'bryg', 'ppłk', 'mar', 'marsz', 'rez', 'ppor', 'DPanc', 'BPanc', 'DKaw', 'p.uł']

extra_abbreviations= extra_abbreviations + position_abbrev + roman_abbrev + quantity_abbrev + place_abbrev + actions_abbrev + place_abbrev + lang_abbrev+military_abbrev

sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

abbrev_set_curr = sentence_tokenizer._params.abbrev_types.copy()
#update abbrev
sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)
abbrev_set_upd = sentence_tokenizer._params.abbrev_types.copy()

# print(f'\ncurrent abbrev={abbrev_set_curr}')
sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)
# print(f'\nupdated abbrev={abbrev_set_upd}')

# print(f'symmetric diff= {abbrev_set_upd.symmetric_difference(abbrev_set_curr)}')

# print(f'Updatad diff= {abbrev_set_upd.difference(abbrev_set_curr)}')
# print(f'Curr diff= {abbrev_set_curr.difference(abbrev_set_upd)}')



In [4]:
files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
    './data/corpus_books_2020_02_24.txt'
]
files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
]
files_to_proces = [
    './data/corpus_books_2020_02_24_fix.txt'
]

from nltk.tokenize import sent_tokenize

for input_file in files_to_proces:
    print(input_file)
    p = Path(input_file)
    output_path = f"{p.with_suffix('')}_lines.txt"

    print(f"in file={input_file}\nout file={output_path}")

    t0=dt.datetime.now()

    total_lines = get_num_lines(input_file)
    
    text=''
    with open(output_path, 'w+') as output_file:
        with open(input_file) as f:
            i=0
            for line in tqdm(f,total=total_lines):

                # get block of text to new line which splits ariticles
                text+=line
                i+=1
                if line.strip() == '' | i%100==0:

                    sentences = sentence_tokenizer.tokenize(text)

                    file_content = ''
                    for sentence in sentences:
                        file_content += sentence.strip()
                        file_content+='\n'
                    output_file.write(file_content)
                    
                    output_file.write('\n')
                    text=''
                    print(f'{i}/{total_lines}={i/total_lines}')

                ## old way
                # if line.strip() == '':
                #     output_file.write('\n')
                #     continue;

                # sentences = sentence_tokenizer.tokenize(line)
                
                # text = ''
                # for sentence in sentences:
                #     text += sentence.strip()
                #     text+='\n'
                # output_file.write(text)
                
    t1=dt.datetime.now()  
    print(f'Split lines done, takes={t1-t0}')   

./data/corpus_books_2020_02_24_fix.txt
in file=./data/corpus_books_2020_02_24_fix.txt
out file=data/corpus_books_2020_02_24_fix_lines.txt


### Train the BPE sentencepiece model

We used the [sentencepiece](https://github.com/google/sentencepiece)


This can be done in two ways:
- as a python module
- as a command line 


If you want to use it as a commandline you should [build the C++ version from source](https://github.com/google/sentencepiece#c-from-source)


Lower txt file 

```
tr [:upper:] [:lower:] < corpus_books_wiki_12M_lines.txt > corpus_books_wiki_12M_lines_lower.txt
```



Train wikipedia vocab using commandline

```
spm_train \
    --input=./data/corpus_wikipedia_2020-02-01_lines.txt \
    --max_sentence_length=4192\
    --model_prefix=./data/wikipedia_v32k_sen10M.spm.bpe \
    --vocab_size=32000 \
    --model_type=bpe \
    --shuffle_input_sentence=true \
    --input_sentence_size=10000000 \
    --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3
```



```
spm_train \
    --input=corpus_books_wiki_12M_lines_lower.txt \
    --max_sentence_length=4192\
    --model_prefix=books_wikipedia_lower_v32k_sen10M.spm.bpe \
    --vocab_size=32000 \
    --model_type=bpe \
    --shuffle_input_sentence=true \
    --input_sentence_size=10000000 \
    --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3
```


In [9]:
import sentencepiece as spm
vocab_size = 32000
model_type = "bpe"  
iss = 10_000_000

vocab_type = 'upper' 
#vocab_type = 'lower' 


tok_model = f"wikipedia_{vocab_type}_voc_{vocab_size}_sen{iss}"

data_file = './data/corpus_wikipedia_2020-02-01_lines.txt'

tok_model = os.path.abspath(f"./data/{tok_model}")

piece_options = ' --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3 --shuffle_input_sentence=true'


cmd = f"--input={data_file} --model_prefix={tok_model} --model_type={model_type} --num_threads=4 --vocab_size={vocab_size}  --input_sentence_size={iss}"+ piece_options
print(cmd)

start = dt.datetime.now()
print(start)
spm.SentencePieceTrainer.train(cmd)
end = dt.datetime.now()

print(f"Vocab of {vocab_size} tokens from {data_file} create takes {end-start}")





--input=./data/corpus_wikipedia_2020-02-01_lines.txt --model_prefix=/home/ksopyla/dev/ermlab/herbert/data/wikipedia_upper_voc_32000_sen10000000 --model_type=bpe --num_threads=4 --vocab_size=32000  --input_sentence_size=10000000 --bos_id=0 --eos_id=1 --pad_id=2 --unk_id=3 --shuffle_input_sentence=true
2020-02-27 13:11:27.769435
Vocab of 32000 tokens from ./data/corpus_wikipedia_2020-02-01_lines.txt create takes 0:13:14.934491


In [10]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load(f"{tok_model}.model")

# returns vocab size
print(sp.get_piece_size())
text = """Będąc młodym programistą (hoho), czytałem "Dziady" w 1983r."""

# encode: text => id
if vocab_type=="lower":
    text = text.lower()

print(sp.encode_as_pieces(text))

32000
['▁Będąc', '▁młodym', '▁programi', 'stą', '▁(', 'ho', 'ho', '),', '▁czy', 't', 'ałem', '▁"', 'D', 'zia', 'dy', '"', '▁w', '▁1983', 'r', '.']


Change the bpe sentecepiece vocab separator form \t to space. It is format needed by fairseq-preprocess 
Open vocab file and replce \t 

In [12]:
 
vocab_file = './data/wikipedia_upper_voc_32000_sen10000000.vocab'


p = Path(vocab_file)

output_path = f"{p.with_suffix('')}_fair.vocab"
with open(output_path, 'w+') as output_file:
    with open(vocab_file) as f:
        
        text = f.read().replace('\t',' ')
        output_file.write(text)


### Split corpus into train, valid, test datasets


Split **corpus_wikipedia_2020-02-01.txt**

* train - 0 -3500002
* valid - 3500002 - 3950001
* test - last - 3950001 - 4355333

```
head -3500002 corpus_wikipedia_2020-02-01.txt > wiki_model/corpus_wikipedia_2020-02-01_train.txt
tail -$((4355333-3950001))  corpus_wikipedia_2020-02-01.txt > wiki_model/corpus_wikipedia_2020-02-01_test.txt

head -3950001 corpus_wikipedia_2020-02-01.txt > corpus_wikipedia_2020-02-01_valid_part.txt
tail -$((3950001-3500002))  corpus_wikipedia_2020-02-01_valid_part.txt > wiki_model/corpus_wikipedia_2020-02-01_valid.txt
rm corpus_wikipedia_2020-02-01_valid_part.txt

```



### Encode data with sentence piece model


Encode wiki 
```
# Encode Data with SentencePiece Tokenizer
cd data/wiki_model

for SPLIT in train valid test; do \
    spm_encode \
    --model=./vocab/wikipedia_upper_voc_32000_sen10000000.model \
    --extra_options=bos:eos \
    --output_format=piece \
    < corpus_wikipedia_2020-02-01_${SPLIT}.txt \
    > corpus_wikipedia_2020-02-01_${SPLIT}.txt.bpe
done
```


### Preprocess data with fairseq

In [1]:

!pipenv install fairseq


Installing fairseq...
Adding fairseq to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (d640ed)...



Fairseq wiki preprocess bpe encoded and splited data

```
fairseq-preprocess \
    --only-source \
    --srcdict ./vocab/wikipedia_upper_voc_32000_sen10000000_fair.vocab \
    --trainpref corpus_wikipedia_2020-02-01_train.txt.bpe \
    --validpref corpus_wikipedia_2020-02-01_valid.txt.bpe \
    --testpref corpus_wikipedia_2020-02-01_test.txt.bpe \
    --destdir ./ \
    --workers 3
```

## Train the model





### AWS production settings

#### p3 instance - linear decay


```
TOTAL_UPDATES=125000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test \
    --save-dir $SAVE_DIR --tensorboard-logdir $LOGS_DIR --keep-last-epochs 10

```

#### g4 instance - wiki triangular


```
TOTAL_UPDATES=125000    # Total number of training steps
STEP_SIZE=5000
BASE_LR=0.0001
PEAK_LR=0.001          # Peak learning rate, adjust as needed
LR_SHRINK=0.6           # better is 0.9
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x


DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_tri
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_tri/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $PEAK_LR \
    --lr-period-updates $STEP_SIZE --lr-shrink $LR_SHRINK --shrink-min \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR \
    --ddp-backend=no_c10d
```

#### g4 instance - wiki cosine



```
TOTAL_UPDATES=30000    # Total number of training steps
WARMUP_UPDATES=1000
BASE_LR=0.00005
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x

DATA_DIR=/mnt/efs/fs1/bert_model/datasets/roberta_data/wiki_model/
SAVE_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_cos
LOGS_DIR=/mnt/efs/fs1/bert_model/checkpoints/wiki_model_cos/logs/

fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler cosine --warmup-updates $WARMUP_UPDATES --lr-period-updates 1000 --t-mult 2 --lr-shrink 0.9 --lr $BASE_LR --max-lr $PEAK_LR \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --skip-invalid-size-inputs-valid-test \
    --tensorboard-logdir $LOGS_DIR --log-format simple --log-interval 1  --save-dir $SAVE_DIR\
    --ddp-backend=no_c10d
```

### Odra settings

```
TOTAL_UPDATES=30000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=6        
UPDATE_FREQ=16          

DATA_DIR=/mnt/hdd/goodwrite_nas/bert_model/datasets/roberta_data/wiki_model

fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 


TOTAL_UPDATES=50000    # Total number of training steps
WARMUP_UPDATES=5000
PEAK_LR=0.001          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=6        
UPDATE_FREQ=16          

DATA_DIR=/mnt/hdd/goodwrite_nas/bert_model/datasets/roberta_data/wiki_model


#after 32184 steps, 12 epoch
fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --reset-lr-scheduler \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 



fairseq-train $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --reset-lr-scheduler \
    --lr-scheduler triangular --lr $BASE_LR --max-lr $MAX_LR \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --max-sentences $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1 --skip-invalid-size-inputs-valid-test --tensorboard-logdir ./logs 